# LDA topic modeling with a curated dataset

This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model.

Depending on your collection size, this example will take between 10 minutes to an hour+ to run. 

In [1]:
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pyLDAvis.gensim

pyLDAvis.enable_notebook()

stop_words = set(stopwords.words('english'))

Initialize a dataset object. 

In [3]:
from tdm_client import Dataset

dset = Dataset('59c090b6-3851-3c65-e016-9181833b4a2c')

In [4]:
len(dset)

6687

In [5]:
dset.query()

'q=*%3A*&fq=yearPublished%3A%5B1700%20TO%202019%5D&fq=-provider%3Aportico&fq=isPartOf%3A(%22Shakespeare%20Quarterly%22)'

Exception in thread tdm-download:
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "//anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "//anaconda3/lib/python3.7/site-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "//anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "//anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  Fi

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [ ]:
def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the volumes in the dataset and make a list of tokens for each volume and then add to a list of the 25 documents in the dataset. We are limiting this example to 25 documents to limit the time it takes to run during demonstrations.

In [ ]:
documents = []

for n, unigram_count in enumerate(dset.get_features()):
    this_doc = []
    for token, count in unigram_count.items():
        clean_token = process_token(token)
        if clean_token is None:
            continue
        this_doc += [clean_token] * count
    documents.append(this_doc)
    if n >= 24:
        break
                

Build a gensim dictionary and corpus and then train the model.

In [ ]:
dictionary = gensim.corpora.Dictionary(documents)

In [ ]:
dictionary.filter_extremes(no_below=len(documents) * .10, no_above=0.5)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]


# train model, this might take some time
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=10,
    passes=15
)


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/).

In [ ]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)